# Test Bed Sliders

Import the test bed from the previous module into a new coordinate system

In [1]:
import bumblebee as bb
import ipywidgets as widgets
from GHelpers import GPlotter

app = bb.Application()
assm = app.csys
assm.ax.set_xlim([-20,20])
assm.ax.set_ylim([-20,20])
assm.ax.set_zlim([0,40])

test_bed = bb.RigidCollection.from_json('TestBedAssm.json')
assm.add(test_bed)

### Set up body variables

In [2]:
test_bed.translate([0,0,15])

In [3]:
arm=None
slide_assm=None
gantry=None
stewart=None
bplate=None
tplate=None

for node in test_bed.nodes:
    arm = node if node.name == 'Arm' else arm
    slide_assm = node if node.name == 'SlideAssm' else slide_assm
    
for node in slide_assm.nodes:
    gantry = node if node.name == 'Gantry' else gantry
    stewart = node if node.name == 'StewartAssm' else stewart
    
for node in stewart.nodes:
    bplate = node if node.name == 'BottomPlateAssm' else bplate
    tplate = node if node.name == 'TopPlateAssm' else tplate

### Stewart Kinematics

In [4]:
bottom_joints = []
for node in bplate.nodes:
    if "Bracket" in node.name:
        for subnode in node.nodes:
            if "Joint" in subnode.name:
                bottom_joints.append(subnode)

In [5]:
top_joints = []
for node in tplate.nodes:
    if "Bracket" in node.name:
        for subnode in node.nodes:
            if "Joint" in subnode.name:
                top_joints.append(subnode)   

In [6]:
import numpy as np
links = []
for bj, tj in zip([0,1,2,3,4,5],[2,5,0,3,4,1]):
    start = top_joints[tj].tf[0:3,3]
    end = bottom_joints[bj].tf[0:3,3]
    
    comb = np.vstack((start,end)).T
    
    link, = assm.ax.plot(*comb, 'r')
    links.append(link)

### Implement Slider Updates

In [7]:
def update_links():
    for i, (bj, tj) in enumerate(zip([0,1,2,3,4,5],[2,5,0,3,4,1])):
        start = top_joints[tj].tf[0:3,3]
        end = bottom_joints[bj].tf[0:3,3]

        comb = np.vstack((start,end)).T

        links[i]._verts3d = (comb[0], comb[1], comb[2])
        
def IK(x,y,z):
    slide_assm.tf[0,3] = x
    slide_assm.update()
    
    bplate.tf[1:3,3] = [y,z]
    bplate.update()
    
    update_links()
    
    assm.fig.canvas.draw()

## Plot!

In [8]:
output = widgets.Output()
with output:
    plotter = GPlotter('TestPrint.gcode', fig=assm.fig, ax=assm.ax, units='in')
    
layer_slider, step_slider = plotter.get_sliders(IK=IK, draw=False)

button = widgets.Button(description='Step Forward')
def on_button_clicked(_):
    plotter.step()
button.on_click(on_button_clicked)

widgets.VBox([widgets.HBox([assm.plot(new=False), layer_slider]), widgets.HBox([step_slider, button])])